# AES workflow

當使用AES（高級加密標準）加密算法時，通常遵循以下工作流程：

## AES加密工作流程

1. **密鑰擴展（Key Expansion）**：
   - 使用初始密鑰，通過密鑰擴展算法生成一系列輪密鑰（Round Keys），以供後續的輪密鑰加操作使用。

2. **初始輪密鑰加（Initial Round Key Addition）**：
   - 使用初始輪密鑰，將明文與初始輪密鑰進行按位異或操作（XOR）。

3. **多輪加密（Multiple Rounds Encryption）**：
   - 重覆執行以下步驟：
     - **字節代替（SubBytes）**：使用S盒（Substitution Box）替換每個字節，增加非線性性。
     - **行移位（ShiftRows）**：按照規定的規則對每一行進行循環左移，實現行內數據混淆。
     - **列混淆（MixColumns）**：將每一列視為多項式，與一個固定的多項式進行乘法運算，增加數據的擴散性。
     - **輪密鑰加（Round Key Addition）**：使用當前輪密鑰與狀態矩陣進行按位異或操作（XOR）。

4. **最終加密輪**：
   - 在最後一輪執行字節代替、行移位和輪密鑰加操作，但不執行列混淆操作。

5. **輸出加密結果**：
   - 將加密後的狀態矩陣轉換為一維數組，並輸出為密文。

## AES解密工作流程

1. **密鑰擴展（Key Expansion）**：
   - 同樣使用初始密鑰，通過密鑰擴展算法生成一系列輪密鑰。

2. **初始輪密鑰加（Initial Round Key Addition）**：
   - 使用初始輪密鑰，將密文與初始輪密鑰進行按位異或操作（XOR）。

3. **多輪解密（Multiple Rounds Decryption）**：
   - 重覆執行以下步驟（與加密步驟相反執行）：
     - **逆行移位（Inverse ShiftRows）**：按照逆規則對每一行進行循環右移，與加密過程相對應。
     - **逆字節代替（Inverse SubBytes）**：使用逆S盒替換每個字節，與加密過程相對應。
     - **輪密鑰加（Round Key Addition）**：使用當前輪密鑰與狀態矩陣進行按位異或操作（XOR）。

4. **最終解密輪**：
   - 在最後一輪執行逆字節代替、逆行移位和輪密鑰加操作，但不執行列混淆操作。

5. **輸出解密結果**：
   - 將解密後的狀態矩陣轉換為一維數組，並輸出為明文。

## 密鑰長度和輪數

- AES支持多種密鑰長度：128位、192位和256位，對應的輪數分別為10輪、12輪和14輪。
- 密鑰長度決定了加密算法的安全性和強度，而輪數決定了加密過程的覆雜度和疊代次數。

這就是AES加密算法的工作流程，涵蓋了加密和解密過程。


# Example of AES encryption